In [4]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

In [8]:
hex_df = gpd.read_file('hexgrid.shp')
contour_vals = pd.read_csv('contour_points_vals.csv')

In [9]:
geometry = [Point(xy) for xy in zip(contour_vals.lon, contour_vals.lat)]
contour_vals = contour_vals.drop(['lon', 'lat'], axis=1)
crs = {'init': 'epsg:4326'}
contour_df = gpd.GeoDataFrame(contour_vals, crs=crs, geometry=geometry)
contour_df.head()

,val,geometry
0,1,POINT (-87.68935700000016 41.534823)
1,1,POINT (-87.68932300000019 41.534823)
2,1,POINT (-87.68911800000004 41.534823)
3,1,POINT (-87.68903300000021 41.534823)
4,1,POINT (-87.68898700000014 41.534823)


In [15]:
hex_join_df = gpd.sjoin(contour_df, hex_df, how="inner", op='within')
hex_join_df.head()

,val,geometry,index_right,bottom,left,right,top
0,1,POINT (-87.68935700000016 41.534823),14738,41.532519,-87.693732,-87.687959,41.537519
1,1,POINT (-87.68932300000019 41.534823),14738,41.532519,-87.693732,-87.687959,41.537519
2,1,POINT (-87.68911800000004 41.534823),14738,41.532519,-87.693732,-87.687959,41.537519
3,1,POINT (-87.68903300000021 41.534823),14738,41.532519,-87.693732,-87.687959,41.537519
4,1,POINT (-87.68898700000014 41.534823),14738,41.532519,-87.693732,-87.687959,41.537519


In [18]:
hex_df.reset_index(inplace=True)
hex_df.head()

,index,bottom,geometry,left,right,top
0,0,41.465019,POLYGON ((-88.10509445396524 41.46751933421439...,-88.105094,-88.099321,41.470019
1,1,41.470019,POLYGON ((-88.10509445396524 41.47251933654564...,-88.105094,-88.099321,41.475019
2,2,41.475019,POLYGON ((-88.10509445396524 41.47751933887689...,-88.105094,-88.099321,41.480019
3,3,41.480019,POLYGON ((-88.10509445396524 41.48251934120814...,-88.105094,-88.099321,41.485019
4,4,41.485019,POLYGON ((-88.10509445396524 41.48751934353939...,-88.105094,-88.099321,41.490019


In [19]:
group_hex_join = hex_join_df.groupby('index_right')['val'].sum()
group_hex_join.head()

index_right
903      9
904      3
906      2
1032    10
1033    13
Name: val, dtype: int64

In [20]:
grp_hex_join_df = pd.DataFrame(group_hex_join).reset_index()
grp_hex_join_df.head()

,index_right,val
0,903,9
1,904,3
2,906,2
3,1032,10
4,1033,13


In [21]:
hex_merge_df = hex_df.merge(grp_hex_join_df, left_on='index', right_on='index_right', how='left')
hex_merge_df.head()

,index,bottom,geometry,left,right,top,index_right,val
0,0,41.465019,POLYGON ((-88.10509445396524 41.46751933421439...,-88.105094,-88.099321,41.470019,NaN,NaN
1,1,41.470019,POLYGON ((-88.10509445396524 41.47251933654564...,-88.105094,-88.099321,41.475019,NaN,NaN
2,2,41.475019,POLYGON ((-88.10509445396524 41.47751933887689...,-88.105094,-88.099321,41.480019,NaN,NaN
3,3,41.480019,POLYGON ((-88.10509445396524 41.48251934120814...,-88.105094,-88.099321,41.485019,NaN,NaN
4,4,41.485019,POLYGON ((-88.10509445396524 41.48751934353939...,-88.105094,-88.099321,41.490019,NaN,NaN


In [22]:
hex_merge_df.val.max()

705.0

In [23]:
hex_merge_df['val'] = hex_merge_df['val'].fillna(0)
hex_merge_df.head()

,index,bottom,geometry,left,right,top,index_right,val
0,0,41.465019,POLYGON ((-88.10509445396524 41.46751933421439...,-88.105094,-88.099321,41.470019,NaN,0.0
1,1,41.470019,POLYGON ((-88.10509445396524 41.47251933654564...,-88.105094,-88.099321,41.475019,NaN,0.0
2,2,41.475019,POLYGON ((-88.10509445396524 41.47751933887689...,-88.105094,-88.099321,41.480019,NaN,0.0
3,3,41.480019,POLYGON ((-88.10509445396524 41.48251934120814...,-88.105094,-88.099321,41.485019,NaN,0.0
4,4,41.485019,POLYGON ((-88.10509445396524 41.48751934353939...,-88.105094,-88.099321,41.490019,NaN,0.0


In [25]:
hex_merge_df.drop('index', axis=1, inplace=True)
hex_merge_df.drop('index_right', axis=1, inplace=True)
hex_merge_df.head()

,bottom,geometry,left,right,top,val
0,41.465019,POLYGON ((-88.10509445396524 41.46751933421439...,-88.105094,-88.099321,41.470019,0.0
1,41.470019,POLYGON ((-88.10509445396524 41.47251933654564...,-88.105094,-88.099321,41.475019,0.0
2,41.475019,POLYGON ((-88.10509445396524 41.47751933887689...,-88.105094,-88.099321,41.480019,0.0
3,41.480019,POLYGON ((-88.10509445396524 41.48251934120814...,-88.105094,-88.099321,41.485019,0.0
4,41.485019,POLYGON ((-88.10509445396524 41.48751934353939...,-88.105094,-88.099321,41.490019,0.0


In [26]:
hex_merge_df.to_file('iso_hex.geojson', driver='GeoJSON')